In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 86.5 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849827 sha256=bb44f59a0bb44fed0b407f2fc4ff610c059013b7aa0cc25c4dd6b127829eb97b
  Stored in directory: /home/codespace/.cache/pip/wheels/13/92/64/da92a3521323cc629fdf25dd56eb26938e08014c1b57ad3759
Successfully built pyspark

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
from pyspark import SparkConf, SparkContext

#On ajoute cette petite condition car Spark ne permet pas de créer plusieurs instances de SparkContext dans le même environnement.
#Il faut donc s'assurer d'arreter un SparkContext existant avant d'en créer un nouveau, si il existe

if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

conf = SparkConf().setAppName("TP_Spark").setMaster("local[*]")
sc = SparkContext(conf=conf)

25/01/29 13:08:43 WARN Utils: Your hostname, codespaces-6a6368 resolves to a loopback address: 127.0.0.1; using 10.0.0.253 instead (on interface eth0)
25/01/29 13:08:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/29 13:08:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/01/29 13:08:55 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### 1.Changer le fichier texte hamlet.txt en tant que spark rdd.

In [9]:
rdd_hamlet =sc.textFile('/workspaces/framework-kafka/hamlet.txt')


### 2.Afficher le type de la variable

In [11]:
rdd_hamlet

/workspaces/framework-kafka/hamlet.txt MapPartitionsRDD[8] at textFile at NativeMethodAccessorImpl.java:0

On voit que cette opération est un MapPartitions

### 3.Afficher la premiere ligne du fichier.

In [12]:
rdd_hamlet.take(1)

['Hamlet by William Shakespeare']

### 4.Combien y a-t-il de ligne dans le fichier ?

In [13]:
rdd_hamlet.count()

632

### 5.Determiner le nombre de lignes qui contiennent le mot 'Hamlet' en utilisant la fonction 'filter'

In [32]:
nb_line_w_hamlet = rdd_hamlet.filter(lambda line: 'Hamlet' in line).count()
print('Voici le nombre de ligne qui continnent le mot Hamlet: ',nb_line_w_hamlet)

Voici le nombre de ligne qui continnent le mot Hamlet:  16


Pour aller plus loin et être sur que ça me trouve tous les Hamlet j'aurais pu tout mettre en majuscule. 

J'ai quand même tester avec Hamlet écris en minuscule il y a 0 résultat.

### 6.Afficher toute les lignes contenant le mot 'twinkle'

In [39]:
line_w_twinkle = rdd_hamlet.filter(lambda line: 'twinkle' in line)
line_w_twinkle.take(-1)

[]

Je ne vois aucun résultat je pense que j'ai due mal écrire le mot. 

J'aurais peut être due faire un rdd_upper mais je n'ai plus le temps

print(rdd_upper.collect())

### 7.calculer et afficher pour chaque mots sa longueur 

In [31]:
word_lengths = (
    rdd_hamlet.flatMap(lambda x:x.split(' '))       #flatMap pour changer voir les mots dans mes lignes grâce à la séparation
    .map(lambda word: (word, len(word)))
)

# Afficher le résultat
print("Longueur des mots :", word_lengths.collect())

Longueur des mots : [('Hamlet', 6), ('by', 2), ('William', 7), ('Shakespeare', 11), ('Characters', 10), ('in', 2), ('the', 3), ('Play', 4), ('======================', 22), ('THE', 3), ('GHOST', 5), ('HAMLET,', 7), ('Prince', 6), ('of', 2), ('Denmark,', 8), ('son', 3), ('of', 2), ('the', 3), ('late', 4), ('King', 4), ('Hamlet', 6), ('and', 3), ('Queen', 5), ('Gertrude', 8), ('QUEEN', 5), ('GERTRUDE,', 9), ('widow', 5), ('of', 2), ('King', 4), ('Hamlet,', 7), ('now', 3), ('married', 7), ('to', 2), ('Claudius', 8), ('KING', 4), ('CLAUDIUS,', 9), ('brother', 7), ('to', 2), ('the', 3), ('late', 4), ('King', 4), ('Hamlet', 6), ('OPHELIA', 7), ('LAERTES,', 8), ('her', 3), ('brother', 7), ('POLONIUS,', 9), ('father', 6), ('of', 2), ('Ophelia', 7), ('and', 3), ('Laertes,', 8), ('councillor', 10), ('to', 2), ('King', 4), ('Claudius', 8), ('REYNALDO,', 9), ('servant', 7), ('to', 2), ('Polonius', 8), ('HORATIO,', 8), ("Hamlet's", 8), ('friend', 6), ('and', 3), ('confidant', 9), ('Courtiers', 9), (

### 8.Calculer les occurences pour chaque mots.

In [30]:
word_occurrences = (
    rdd_hamlet.flatMap(lambda line: line.split(' '))
       .map(lambda word: (word, 1))  
       .reduceByKey(lambda a, b: a + b)    
)

print("Occurrences des mots :", word_occurrences.collect())

Occurrences des mots : [('by', 13), ('Shakespeare', 1), ('GHOST', 1), ('HAMLET,', 2), ('of', 80), ('and', 82), ('widow', 1), ('married', 2), ('to', 84), ('Claudius', 2), ('KING', 8), ('CLAUDIUS,', 1), ('brother', 2), ('OPHELIA', 1), ('her', 4), ('father', 7), ('Polonius', 1), ("Hamlet's", 1), ('friend', 2), ('confidant', 1), ('Courtiers', 1), ('at', 6), ('Danish', 2), ('court:', 1), ('', 171), ('VOLTEMAND', 1), ('ROSENCRANTZ', 1), ('OSRIC', 1), ('A', 10), ('FRANCISCO', 9), ('MARCELLUS', 17), ('FORTINBRAS,', 1), ('army', 1), ('from', 6), ('take', 4), ('roles', 1), ('Player', 2), ('King,', 1), ('Lucianus', 1), ('Murder', 1), ('Sailors', 1), ("Gravedigger's", 1), ('Doctor', 1), ('Attendants,', 1), ('Guards,', 1), ('Musicians,', 1), ("Laertes's", 1), ('Soldiers,', 1), ('Officers', 1), ('=====', 1), ('=======', 2), ('Barnardo', 3), ('sentinels.]', 1), ("Who's", 1), ('there?', 3), ('answer', 1), ('me.', 3), ('yourself.', 2), ('Long', 1), ('live', 1), ('come', 3), ('most', 13), ('hour.', 1), 